# Perform regression on parameterized surfaces

## Imports

In [7]:
# imports

import trimesh
import os
import sys
import subprocess
import torch
import numpy as np
os.environ["GEOMSTATS_BACKEND"] = "pytorch"
import geomstats.backend as gs
import my28brains.discrete_surfaces

In [8]:
gitroot_path = subprocess.check_output(
    ["git", "rev-parse", "--show-toplevel"], universal_newlines=True
)
os.chdir(gitroot_path[:-1])
work_dir = os.getcwd()

my28brains_dir = os.path.join(work_dir, "my28brains")

sys_dir = os.path.dirname(work_dir)
sys.path.append(sys_dir)
sys.path.append(my28brains_dir)

# this forces the notebook to re-load
import importlib
importlib.reload(my28brains.discrete_surfaces)

from my28brains.discrete_surfaces import (
    DiscreteSurfaces,
    ElasticMetric,
)

import my28brains.default_config as default_config

## Generate parameterized geodesic from sphere to ellipse using `discrete_curves.py`.

In [18]:
#create sphere mesh
sphere = trimesh.creation.icosphere(subdivisions=3, radius=30.0)

#Create ellipsoid mesh
# Create a scaling matrix for the semi-axes lengths
scales = np.array([2, 2, 3])
scale_matrix = np.diag(scales)

scale_matrix = gs.array(scale_matrix)

# Apply the scaling transformation to the mesh vertices
scaled_vertices = sphere.vertices.dot(scale_matrix)

# Create a new mesh with the scaled vertices
ellipsoid = trimesh.Trimesh(vertices=scaled_vertices, faces=sphere.faces)

#Define Surface Space
SURFACE_SPACE = DiscreteSurfaces(faces = sphere.faces)

METRIC = ElasticMetric(
    space = SURFACE_SPACE,
    a0=default_config.a0,
    a1=default_config.a1,
    b1=default_config.b1,
    c1=default_config.c1,
    d1=default_config.d1,
    a2=default_config.a2,)

# CREATE GEODESIC
n_vertices = sphere.vertices.shape[0]
print(n_vertices)

def geodesics_sphere_to_ellipsoid(
    n_geodesics=1, n_times=5
):
    """Generate a dataset of geodesics that transform spheres into ellipsoids."""
    dim = 3
    
    geodesics = gs.zeros((n_geodesics, n_times, n_vertices, dim))
    times = gs.arange(0, 1, 1 / n_times)
    for i_geodesic in range(n_geodesics):
        start_surface = gs.array(sphere.vertices)
        end_surface = gs.array(ellipsoid.vertices)
        print(start_surface.shape)
        geodesic = METRIC.geodesic(initial_point = start_surface, end_point = end_surface)
        
        geodesics[i_geodesic] = geodesic
        # geodesics[i_geodesic] = geodesic(times)

    return geodesics

geodesics = geodesics_sphere_to_ellipsoid()
geodesic = geodesics[0]

642
torch.Size([642, 3])
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =         5778     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  9.36869D+07    |proj g|=  2.31308D+03

At iterate    1    f=  9.36708D+07    |proj g|=  3.97712D+03

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
 5778      1      2      1     0     0   3.977D+03   9.367D+07
  F =   93670813.262415290     

CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH             


 This problem is unconstrained.


# GEODESIC REGRESSION

In [19]:
# Regression imports

import matplotlib.pyplot as plt

import geomstats.backend as gs
import geomstats.visualization as visualization
from geomstats.geometry.special_euclidean import SpecialEuclidean
from geomstats.learning.frechet_mean import FrechetMean, variance
from geomstats.learning.geodesic_regression import GeodesicRegression

In [20]:
gr = GeodesicRegression(
        SURFACE_SPACE,
        metric=METRIC,
        center_X=False,
        method="riemannian",
        max_iter=100,
        init_step_size=0.1,
        verbose=True,
        initialization="frechet",
    )

X = gs.random.normal(size=(5))
X -= gs.mean(X)
print(geodesic.shape)

gr.fit(X, geodesic, compute_training_score=True)

intercept_hat, beta_hat = gr.intercept_, gr.coef_

torch.Size([5, 642, 3])


IndexError: index 532 is out of bounds for axis 0 with size 5